In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('dark')

In [ ]:
china = pd.read_excel('Brazil_China.xlsx',skiprows=12)

In [ ]:
china = china.iloc[:, 0:20]

In [ ]:
china.shape

In [ ]:
china.columns

In [ ]:
china = china.fillna('Agriculture')

In [ ]:
china = china.set_index(['Product code', 'Product label'])

In [ ]:
china = china.transpose()

In [ ]:
china.reset_index(level=0, inplace=True)

In [ ]:
china = china.rename(index= str, columns = {'index':'year'})

In [ ]:
china.columns

In [ ]:
china['year'] = china['year'].replace({'Value in 2001': '2001', 'Value in 2002': '2002', 'Value in 2003':'2003', 'Value in 2004':'2004',
                            'Value in 2005':'2005', 'Value in 2006':'2006', 'Value in 2007':'2007', 'Value in 2008':'2008', 
                            'Value in 2009':'2009', 'Value in 2010':'2010', 'Value in 2011':'2011', 'Value in 2012':'2012',
                            'Value in 2013': '2013', 'Value in 2014':'2014', 'Value in 2015':'2015', 'Value in 2016':'2016', 
                            'Value in 2017':'2017', 'Value in 2018':'2018'})

In [ ]:
from datetime import datetime
from dateutil.parser import parse

In [ ]:
type(china.year)

In [ ]:
china['year'] = pd.to_datetime(china.year, format='%Y')

In [ ]:
import datetime as dt

In [ ]:
china['year'] = china['year'].dt.year

In [ ]:
#china.to_excel("usable_china_data.xlsx")

In [ ]:
# now do the same thing with Africa Data

In [ ]:
africa = pd.read_excel('Brazil_Africa.xlsx',skiprows=12)

In [ ]:
africa = africa.iloc[:, 0:20]

In [ ]:
africa.shape

In [ ]:
africa = africa.fillna('Agriculture')

In [ ]:
africa = africa.set_index(['Product code', 'Product label'])

In [ ]:
africa = africa.transpose()

In [ ]:
africa.reset_index(level=0, inplace=True)

In [ ]:
africa = africa.rename(index= str, columns = {'index':'year'})

In [ ]:
africa['year'] = africa['year'].replace({'Value in 2001': '2001', 'Value in 2002': '2002', 'Value in 2003':'2003', 'Value in 2004':'2004',
                            'Value in 2005':'2005', 'Value in 2006':'2006', 'Value in 2007':'2007', 'Value in 2008':'2008', 
                            'Value in 2009':'2009', 'Value in 2010':'2010', 'Value in 2011':'2011', 'Value in 2012':'2012',
                            'Value in 2013': '2013', 'Value in 2014':'2014', 'Value in 2015':'2015', 'Value in 2016':'2016', 
                            'Value in 2017':'2017', 'Value in 2018':'2018'})

In [ ]:
africa['year'] = pd.to_datetime(africa.year, format='%Y')

In [ ]:
africa['year'] = africa['year'].dt.year

In [ ]:
africa.head()

In [ ]:
chinaprcchng = (china.iloc[:,1:]).pct_change()

In [ ]:
chinaprcchng['year'] = china.year

In [ ]:
chinaprcchng.head()

In [ ]:
africaprcchng = (africa.iloc[:,1:]).pct_change()

In [ ]:
africaprcchng['year'] = africa.year

In [ ]:
africaprcchng[['Agriculture','year']]

In [ ]:
africa_ag_pctchng = africaprcchng[['Agriculture','year']]

In [ ]:
africa_ag_pctchng.to_excel("africa_pctchng.xlsx")

In [ ]:
#setting the year date variable as as the index, but creating it as a new object. This is for the SMA

In [ ]:
china_yindx = china.set_index('year')

In [ ]:
africa_yindx = africa.set_index('year')

In [ ]:
china.index = pd.to_datetime(china_yindx.index, format='%Y')

In [ ]:
africa.index = pd.to_datetime(africa_yindx.index, format='%Y')

In [ ]:
africa.head()

In [ ]:
africa['3-year-SMA']=africa['Agriculture'].rolling(window=3).mean()
africa['6-year-SMA']=africa['Agriculture'].rolling(window=6).mean()

In [ ]:
africa.head()

In [ ]:
africa_ag_SMA = africa.loc[:,['Agriculture','3-year-SMA','6-year-SMA']]

In [ ]:
africa_ag_SMA.plot(figsize = (14,8))

In [ ]:
china.head()

In [ ]:
#china_Ag = pd.DataFrame(china['Agriculture'])

In [ ]:
#africa_Ag = pd.DataFrame(africa['Agriculture'])

In [ ]:
#china_Ag.columns = ['China_Agriculture']

In [ ]:
#africa_Ag.columns = ['Africa_Agriculutre']

In [ ]:
#new_series = pd.concat([china_Ag,africa_Ag])

In [ ]:
#new_series.plot(figsize = (14,8))

In [ ]:
china.columns

In [ ]:
china.head()

In [ ]:
cluster_china_data = china.drop(columns = ['year',"'TOTAL", 'Agriculture'])

In [ ]:
cluster_china_data.head()

In [ ]:
from scipy import stats
import scipy.cluster.hierarchy as hac

In [ ]:
import seaborn as sns

In [ ]:
Z = hac.linkage(cluster_china_data, method='single', metric='correlation')

# Plot dendogram
plt.figure(figsize=(35, 18))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Products')
plt.ylabel('distance')
hac.dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
# Here we use spearman correlation
def my_metric(x, y):
    r = stats.pearsonr(x, y)[0]
    return 1 - r # correlation to distance: range 0 to 2

# Do the clustering    
Z = hac.linkage(cluster_china_data,  method='single', metric=my_metric)

# Plot dendogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Products')
plt.ylabel('distance')
hac.dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import fcluster

def print_clusters(cluster_china_data, Z, k, plot=False):
    # k Number of clusters I'd like to extract
    results = fcluster(Z, k, criterion='maxclust')

    # check the results
    s = pd.Series(results)
    clusters = s.unique()

    for c in clusters:
        cluster_indeces = s[s==c].index
        print("Cluster %d number of entries %d" % (c, len(cluster_indeces)))
        if plot:
            cluster_china_data.T.iloc[:,cluster_indeces].plot()
            plt.show()

print_clusters(cluster_china_data, Z, 6, plot=True)

In [ ]:
cluster_china_data.columns

In [ ]:
#cluster_china_data.reset_index(drop=True, inplace=True) #easy wasy to drop an index evidently

In [ ]:
cluster_china_data.shape

In [ ]:
cluster_china_data.head()

In [ ]:
#cluster_china_data.to_excel("cluster_china.xlsx")